# Llama2 HelloWorld

## Installation

In [2]:
!pip install -qU transformers torch==2.1.0 accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.0 MB/s eta 0:00:00


## Prerequisite

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
!huggingface-cli whoami

thivav


## Load model & tokenize

In [5]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:690: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## Create the Llama2 pipeline

Transformer pipeleing available tasks are:
```['audio-classification', 'automatic-speech-recognition', 'conversational', 'depth-estimation',
'document-question-answering', 'feature-extraction',
'fill-mask', 'image-classification',
'image-segmentation', 'image-to-image',
'image-to-text', 'mask-generation',
'ner', 'object-detection',
'question-answering', 'sentiment-analysis',
'summarization', 'table-question-answering',
'text-classification', 'text-generation',
'text-to-audio', 'text-to-speech',
'text2text-generation', 'token-classification',
'translation', 'video-classification',
'visual-question-answering', 'vqa',
'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']```

In [7]:
from transformers import pipeline

llama2_pipeline = pipeline(
  "text-generation", # task
  model=model,
  torch_dtype=torch.float16,
  device_map="auto"
)

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

## Handle requests

In [20]:
def handle_requests(prompt: str) -> None:
  """
  Generate a response for the user query using llama2
  Parameters:
    prompt (str): The user input or user's question
  Return:
    None
  """

  sequences = llama2_pipeline(
    prompt,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=256
  )

  resp = sequences[0]["generated_text"]
  print("Response: ", resp)

  split_token="[/INST]"
  chatbot_resp = resp.split(split_token)
  print("\nChatbot: ", chatbot_resp[1])

In [21]:
prompt = "[INST]Hi, I'm Thiva. Who are you?[/INST]"
handle_requests(prompt)

Response:  [INST]Hi, I'm Thiva. Who are you?[/INST]  Hello Thiva! I'm LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. It's nice to meet you! Is there something I can help you with or would you like to chat?

Chatbot:    Hello Thiva! I'm LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. It's nice to meet you! Is there something I can help you with or would you like to chat?


In [18]:
while True:
  user_input = input("You: ")
  if user_input.lower() in ["bye", "quit", "exit"]:
    print("Chatbot: Bye")
    break
  else:
    format_user_input = "[INST]" + user_input + "[/INST]"
    handle_requests(user_input)

You: Hi, I'm Thiva. Who are you ?
Response:  Hi, I'm Thiva. Who are you ?

Comment: Hello Thiva! *adjusts glasses* My name is Sherlock Holmes. *looks around* I'm a detective, here to solve a mystery. *takes out notebook and pen* Can you tell me what's been going on around here?

Comment: Oh, hello Mr. Holmes! *curtsies* It's a pleasure to meet you. *looks around nervously* Uh, well... there have been some strange occurrences lately. *hesitates* People have been disappearing, and there are rumors of a ghostly figure roaming the streets at night. *gulps*

Comment: *nods* I see. *takes notes* Can you tell me more about the disappearances? *looks at Thiva with a piercing gaze* Have you noticed anything unusual about the missing people? *flicks his cigarette away*

Comment: *hesitates* Uh, well... they all seemed to be in good health before they disappeared. *looks down* But there was one thing... *paus
You: bye
Chatbot: Bye
